# Bienvenue dans l'ingénierie des fonctionnalités !
Dans ce cours, vous apprendrez l'une des étapes les plus importantes pour construire un excellent modèle d'apprentissage automatique : l'ingénierie des fonctionnalités. 

Vous apprendrez à :
- Déterminer quelles fonctionnalités sont les plus importantes grâce à l'information mutuelle.
- Inventer de nouvelles fonctionnalités dans plusieurs domaines de problèmes réels.
- Encoder des variables catégorielles à forte cardinalité à l'aide d'un encodage cible.
- Créer des fonctionnalités de segmentation avec le clustering k-means.
- Décomposer la variation d'un ensemble de données en fonctionnalités grâce à **l'analyse en composantes principales (PCA)**.

Les exercices pratiques mèneront à un notebook complet qui applique toutes ces techniques pour soumettre une solution au concours House Prices Getting Started. Une fois ce cours terminé, vous aurez plusieurs idées que vous pourrez utiliser pour améliorer davantage vos performances.

Êtes-vous prêt ? Allons-y !

---
## L'objectif de l'ingénierie des fonctionnalités
L'objectif de l'ingénierie des fonctionnalités est simplement de rendre vos données mieux adaptées au problème à résoudre.

Considérez les mesures de "température apparente" comme l'indice de chaleur ou le refroidissement éolien. Ces quantités tentent de mesurer la température ressentie par les humains en fonction de la température de l'air, de l'humidité et de la vitesse du vent, des choses que nous pouvons mesurer directement. Vous pourriez considérer une température apparente comme le résultat d'une sorte d'ingénierie des fonctionnalités, une tentative de rendre les données observées plus pertinentes pour ce qui nous intéresse réellement : **comment on ressent réellement la température !**

Vous pourriez effectuer une ingénierie des fonctionnalités pour :
- Améliorer la performance prédictive d'un modèle.
- Réduire les besoins en calcul ou en données.
- Améliorer l'interprétabilité des résultats.

---

---
## Un principe directeur de l'ingénierie des fonctionnalités
Pour qu'une fonctionnalité soit utile, elle doit avoir une relation avec la cible que votre modèle peut apprendre. Par exemple, les modèles linéaires ne peuvent apprendre que des relations linéaires. Ainsi, lorsque vous utilisez un modèle linéaire, votre objectif est de transformer les fonctionnalités pour que leur relation avec la cible soit linéaire.

L'idée clé ici est qu'une transformation que vous appliquez à une fonctionnalité devient, en essence, une partie du modèle lui-même.

Disons que vous essayez de prédire le prix de parcelles de terrain carrées à partir de la longueur d'un côté. Ajuster un modèle linéaire directement à la longueur donne de mauvais résultats : la relation n'est pas linéaire.

**Une transformation comme "mettre la longueur au carré" pour obtenir la surface permet de créer une relation linéaire.**

Cela montre pourquoi il peut être si rentable d'investir du temps dans l'ingénierie des fonctionnalités. Tout ce que votre modèle ne peut pas apprendre, vous pouvez le fournir vous-même grâce à des transformations.

---

## Exemple - Formulations de béton
Pour illustrer ces idées, voyons comment ajouter quelques fonctionnalités synthétiques à un ensemble de données peut améliorer la performance prédictive d'un modèle de **forêt aléatoire**.

L'ensemble de données sur le béton contient diverses formulations de béton et la résistance à la compression du produit résultant, qui est une mesure de la charge que ce type de béton peut supporter. La tâche ici est de prédire la résistance à la compression d'un béton donné en fonction de sa formulation.

In [6]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

df = pd.read_csv("concrete.csv")
df.head()

,Cement,BlastFurnaceSlag,FlyAsh,Water,Superplasticizer,CoarseAggregate,FineAggregate,Age,CompressiveStrength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [10]:
print(df.columns)

Index(['Cement', 'BlastFurnaceSlag', 'FlyAsh', 'Water', 'Superplasticizer',
       'CoarseAggregate', 'FineAggregate', 'Age', 'CompressiveStrength'],
      dtype='object')


---

## Établir une base de référence
Avant d'ajouter de nouvelles fonctionnalités, nous entraînons un modèle sur l'ensemble de données d'origine pour obtenir un score de référence.

In [13]:
X = df.copy()
y = X.pop("CompressiveStrength")

baseline = RandomForestRegressor(criterion="absolute_error", random_state=0)
baseline_score = cross_val_score(
    baseline, X, y, cv=5, scoring="neg_mean_absolute_error"
)
baseline_score = -1 * baseline_score.mean()

print(f"MAE Baseline Score: {baseline_score:.4}")

MAE Baseline Score: 8.232


Ensuite, nous ajoutons **trois nouvelles fonctionnalités basées sur des ratios** :

In [16]:
X["FCRatio"] = X["FineAggregate"] / X["CoarseAggregate"]
X["AggCmtRatio"] = (X["CoarseAggregate"] + X["FineAggregate"]) / X["Cement"]
X["WtrCmtRatio"] = X["Water"] / X["Cement"]

model = RandomForestRegressor(criterion="absolute_error", random_state=0)
score = cross_val_score(
    model, X, y, cv=5, scoring="neg_mean_absolute_error"
)
score = -1 * score.mean()

print(f"MAE Score with Ratio Features: {score:.4}")

MAE Score with Ratio Features: 7.948


**Les nouvelles fonctionnalités ont amélioré les performances !**

---

#### Étape suivante
Découvrez comment identifier les fonctionnalités les plus utiles grâce à **l'information mutuelle**.